In [1]:
#https://detectron2.readthedocs.io/
import detectron2
import logging
import torch
from detectron2.utils.logger import setup_logger
setup_logger()
logger = logging.getLogger('detectron2')

In [2]:
#Paths
from pathlib import Path
base_folder = Path('.')
data_folder = base_folder/'til2020'
train_imgs_folder = data_folder/'train'
train_annotations = data_folder/'train.json'
val_imgs_folder = data_folder/'val'
val_annotations = data_folder/'val.json'
test_imgs_folder = data_folder/'CV_interim_images'

#keep these the same
save_model_folder = base_folder/'ckpts'
load_model_folder = base_folder/'ckpts'

In [3]:
#register datasets. now they can be used as if they were native. remember to run fix_annotations.py to convert TIL2020 to proper COCO format
#to implement custom loaders, such as pickled, https://detectron2.readthedocs.io/modules/data.html?highlight=DatasetCatalog#detectron2.data.DatasetCatalog
from detectron2.data.datasets import register_coco_instances
register_coco_instances("til_train", {}, train_annotations, train_imgs_folder)
register_coco_instances("til_val", {}, val_annotations, val_imgs_folder)

In [4]:
#https://detectron2.readthedocs.io/modules/config.html
#btw, i added some custom config options for my checkpointer and pipeline
from detectron2 import model_zoo
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
#cfg.SEED = 42

cfg.DATASETS.TRAIN = ("til_train",)
cfg.DATASETS.TEST = ("til_val",)
cfg.OUTPUT_DIR = str(save_model_folder)

cfg.DATALOADER.NUM_WORKERS = 12
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler" #deals with class imbalance
cfg.DATALOADER.REPEAT_THRESHOLD = 0.5

cfg.SOLVER.IMS_PER_BATCH = 2 #batch_size
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.GAMMA = 0.5 #halves the learning rate at each milestone
cfg.SOLVER.STEPS = (10000000,)#(20000,60000,100000,140000) # milestones in iterations
#^I played with these but realised it had little effect: so keep BASE_LR at 0.000125*IMS_PER_BATCH

#Pipeline augmentation settings (i implemented these)
cfg.INPUT.CROP.ENABLED = False
cfg.INPUT.CROP.SIZE = [0.7, 0.9]
#cfg.INPUT.RAND_ROTATION = [-2,2]
cfg.INPUT.RAND_CONTRAST = [0.8,1.2]
cfg.INPUT.RAND_BRIGHTNESS = [0.8,1.2]
cfg.INPUT.RAND_SATURATION = [0.8,1.2]


cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of categories
cfg.PRINT_EVERY = 200
cfg.SAVE_EVERY = 1000 #when using ValCheckpointer, it saves only if val loss is the minimum so far

cfg.SOLVER.MAX_ITER = -1
cfg.SOLVER.EPOCHS = 50 #facebook uses the paradigm of endless datastreams, as such epochs dont really exist
cfg.EPOCH_SIZE = int(8225/cfg.SOLVER.IMS_PER_BATCH)

logger.info(f"Loaded Config:\n{cfg.dump()}")

[06/15 15:47:24 detectron2]: Loaded Config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 12
  REPEAT_THRESHOLD: 0.6
  SAMPLER_TRAIN: RepeatFactorTrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - til_val
  TRAIN:
  - til_train
EPOCH_SIZE: 2741
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: false
    SIZE:
    - 0.8
    - 0.9
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN:
  - 640
  - 672
  - 704
  - 736
  - 768
  - 800
  MIN_SIZE_TRAIN_SAMPLING: choice
  RAND_BRIGHTNESS:
  - 0.95
  - 1.05
  RAND_CONTRAST:
  - 0.95
  - 1.05
  RAND_SATURATION:
  - 0.95
  - 1.05
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: DefaultAnch

In [5]:
from detectron2.modeling import build_model
model = build_model(cfg)
logger.info(f"Created Model:\n{model}")

 (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
        (2): BottleneckBlock(
          (conv1): Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
      )
      (res3): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
          (conv1): Conv2d(
            256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False
            (n

In [7]:
from pipeline import DatasetPipeline

#uses whatever pycocotools is installed, make sure it is the TIL one
def do_test(cfg,model):
    from detectron2.evaluation import COCOEvaluator, inference_on_dataset
    from detectron2.data import build_detection_test_loader
    evaluator = COCOEvaluator(cfg.DATASETS.TEST[0],cfg,False)
    val_loader = build_detection_test_loader(cfg,cfg.DATASETS.TEST[0])
    inference_on_dataset(model,val_loader,evaluator)

def do_eval(cfg,model):
    from detectron2.data import build_detection_test_loader,DatasetMapper
    from tqdm import tqdm
    dataloader = build_detection_test_loader(cfg,cfg.DATASETS.TEST[0],mapper=DatasetPipeline(cfg,False))
    total_loss = 0
    logger.info("Calculating Validation Loss...")
    with torch.no_grad():
        for iteration,data in enumerate(tqdm(dataloader)):
            loss_dict = model(data)
            total_loss += sum(loss_dict.values())
    tqdm.write("\n")
    return total_loss/len(dataloader)

def do_train(cfg,model,model_context,resume=False):
    from detectron2.utils.events import (
        CommonMetricPrinter,
        EventStorage,
        JSONWriter,
        TensorboardXWriter,
    )
    import detectron2.utils.comm as comm
    from detectron2.solver import build_lr_scheduler, build_optimizer
    from detectron2.data import build_detection_train_loader
    from detectron2.checkpoint import DetectionCheckpointer
    from checkpointer import ValCheckpointer #eh, it gets the essence of early stopping & Im too lazy to make it actually early stop

    optimizer = build_optimizer(cfg, model)
    scheduler = build_lr_scheduler(cfg, optimizer)
    saver = DetectionCheckpointer(model, cfg.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler)
    meta = saver.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume) #loads the model weights & returns stored meta

    val_loss = meta.get('min_val_loss',meta.get('val_loss',999))
    checkpointer = ValCheckpointer(saver,cfg.SAVE_EVERY,model_context,lambda: do_eval(cfg,model),val_loss)
    
    if 'EPOCHS' in cfg.SOLVER.keys(): cfg.SOLVER.MAX_ITER = cfg.SOLVER.EPOCHS*cfg.EPOCH_SIZE
    max_iter = cfg.SOLVER.MAX_ITER
    start_iter = meta.get("iteration",-1)+1 

    if resume: 
        logger.info(f"Resumed model: {meta.get('model_name','unknown')}")
        #override some configs from checkpoint in case you changed them
        scheduler.milestones = cfg.SOLVER.STEPS
        scheduler.gamma = cfg.SOLVER.GAMMA
        scheduler.base_lrs = [cfg.SOLVER.BASE_LR for lr in scheduler.base_lrs]
        scheduler.last_epoch = start_iter

    writers = [
        CommonMetricPrinter(max_iter),
        JSONWriter(f"{cfg.OUTPUT_DIR}/{model_context}-metrics.json"),
        TensorboardXWriter(cfg.OUTPUT_DIR),
    ]

    model.train() #set to training mode (PyTorch)
    dataloader = build_detection_train_loader(cfg,mapper=DatasetPipeline(cfg,True))
    logger.info(f"Training: Start Iter {start_iter}, End Iter {max_iter}")
    with EventStorage(start_iter) as storage:
        for iteration,data in zip(range(start_iter,max_iter),dataloader):
            try:
                iteration = iteration + 1
                storage.step()
                
                loss_dict = model(data)
                losses = sum(loss_dict.values())
                assert torch.isfinite(losses).all(), loss_dict

                loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
                losses_reduced = sum(loss for loss in loss_dict_reduced.values())
                if comm.is_main_process(): storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)

                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
                storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
                scheduler.step()

                if iteration - start_iter > 5 and (iteration % cfg.PRINT_EVERY == 0 or iteration == max_iter):
                    for writer in writers: writer.write()
                checkpointer.step(iteration)
                
            except (Exception,KeyboardInterrupt) as e:
                logger.info("ERROR! Dumping current model...")
                checkpointer.save(f"{model_context}-{iteration}-interrupted",iteration=iteration,model_name=model_context,min_val_loss=checkpointer.min_loss)
                raise e
    checkpointer.save(f"{model_context}-{max_iter}-final",iteration=max_iter,model_name=model_context,min_val_loss=checkpointer.min_loss)


In [9]:
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-17999-best_val.pth")
do_train(cfg,model,"ft-til_resnet101_rcnn",resume=True)
#if this doesnt work correctly, check what model is set in the  file in /ckpts/last_checkpoint
#I uploaded the model file to the google drive
#have 40% of the images contain no 'person's to prevent overeagerness. figure out how to filter out 'person' images so you can delete the rest of the enormous coco dataset. 
#^use segmentation model, that way you can remove the background before passing it to the fashion model. How to indicate it is the background tho? put a checkerboard? somehow prevent person/clothes blending into background...

.000250  max_mem: 5317M
[06/15 13:25:04 d2.utils.events]:  eta: 1:48:51  iter: 125800  total_loss: 0.144  loss_cls: 0.050  loss_box_reg: 0.079  loss_rpn_cls: 0.002  loss_rpn_loc: 0.006  lr: 0.000250  max_mem: 5317M
[06/15 13:27:00 detectron2]: Pipeline: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[06/15 13:27:00 d2.data.datasets.coco]: Loaded 1474 images in COCO format from til2020/val.json
[06/15 13:27:00 d2.data.common]: Serializing 1474 elements to byte tensors and concatenating them all ...
[06/15 13:27:00 d2.data.common]: Serialized dataset takes 0.36 MiB
[06/15 13:27:00 detectron2]: Calculating Validation Loss...
100%|██████████| 1474/1474 [02:20<00:00, 10.49it/s]

[06/15 13:29:21 detectron2]: Val Loss: 0.2837391197681427 @ Iteration 125999, Min Val Loss: 0.18455912172794342, did not save model.

[06/15 13:29:21 d2.utils.events]:  eta: 3:56:59  iter: 126000  total_loss: 0.112  loss_cls: 0.036  loss_box_reg: 0.068  loss_rpn_cls: 0.000  

In [8]:
#if do_train() hasnt been called (it loads the model) and you want to load the model
if True:
    from detectron2.checkpoint import DetectionCheckpointer
    cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-17999-best_val.pth")
    DetectionCheckpointer(model, cfg.OUTPUT_DIR).resume_or_load(cfg.MODEL.WEIGHTS, resume=False)

do_test(cfg,model)

[06/15 15:47:53 d2.data.datasets.coco]: Loaded 1474 images in COCO format from til2020/val.json
[06/15 15:47:53 d2.data.build]: Distribution of instances among all 5 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    tops    | 317          |  trousers  | 313          | outerwear  | 316          |
|  dresses   | 1338         |   skirts   | 174          |            |              |
|   total    | 2458         |            |              |            |              |
[06/15 15:47:53 d2.data.common]: Serializing 1474 elements to byte tensors and concatenating them all ...
[06/15 15:47:53 d2.data.common]: Serialized dataset takes 0.36 MiB
[06/15 15:47:53 d2.evaluation.evaluator]: Start inference on 1474 images
[06/15 15:47:56 d2.evaluation.evaluator]: Inference done 11/1474. 0.1629 s / img. ETA=0:03:58
[06/15 15:48:01 d2.evaluation.evaluator]: Inference 

In [5]:
from detectron2.engine import DefaultPredictor
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-143999-best_val.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0 #max AP at cost of inference speed
predictor = DefaultPredictor(cfg) #check what exactly default predictor does, and if it affects anything badly.
#Are output boxes supposed to be rescaled? btw, the current pipeline resizes back to original dimensions

In [6]:
def boxes2xywh(boxes):
    return [[x1,y1,x2-x1,y2-y1] for x1,y1,x2,y2 in boxes.tensor.tolist()]

def convert_prediction(img_id,output):
    outs = []
    raw = output['instances']
    boxes = boxes2xywh(raw.pred_boxes)
    cats = raw.pred_classes.tolist()
    scores = raw.scores.tolist()

    for i in range(len(raw)):
        outs.append({
            'image_id':img_id,
            'category_id':cats[i],
            'bbox':boxes[i],
            'score':scores[i]
        })
    return outs


In [9]:
import json
from tqdm import tqdm
import numpy as np
import cv2

output = []
for im_path in tqdm(list(test_imgs_folder.glob('*'))):
    im = cv2.imread(im_path) #we are using cv2 here to be absolutely sure; this is what is in the documentation
    output += convert_prediction(int(im_path.stem),predictor(im))

with open('ans.json','w') as f:
    json.dump(output,f)

100%|██████████| 1000/1000 [02:44<00:00,  6.07it/s]


In [16]:
import cv2
from IPython.display import display
import numpy as np

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
im = cv2.imread(train_imgs_folder/"69.jpg")
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im_out = Image.fromarray(v.get_image()[:, :, ::-1]) #channels are reversed
#display(im_out)